<a href="https://colab.research.google.com/github/Mathufnn/congresso_artificial/blob/master/interpretador_de_leis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# autentica o uso do drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# descomprime as pastas de arquivos na cache do notebook. nao afeta o google drive

!unzip /content/drive/My\ Drive/congresso_artificial/dataset_treinamento_arquivo.zip
!unzip /content/drive/My\ Drive/congresso_artificial/lista_votacoes_xml.zip

Archive:  /content/drive/My Drive/congresso_artificial/dataset_treinamento_arquivo.zip
replace dataset_treinamento/materia10_pag10_ano2010.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/My Drive/congresso_artificial/lista_votacoes_xml.zip
replace lista_votacoes_xml/lista_votacoes1991.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
# verifica se está usando a GPU 
# saida esperada: '/device:GPU:0'

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import text_to_word_sequence
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

####

directory = '/content/dataset_treinamento/'
raw_text = ""

for filename in reversed(os.listdir(directory)):
    if filename.endswith(".txt"):
      with open(directory + filename, 'r') as f:
        raw_text += f.read() + " "

####


words = text_to_word_sequence(raw_text, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n', split=' ', lower=True)

MAX_WORDS = 100000
words = words[0:MAX_WORDS]

# print("Amostra do texto: ", end="")
# for i in words:
#   print(i + " ", end="")
# print("\n")

dict_words = sorted(set(words))

n_words = len(words)
n_vocab = len(dict_words)
print("Número de palavras: ", n_words)
print("Tamanho do vocabulário: ", n_vocab)

In [5]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import text_to_word_sequence
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

seq_length = 50
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
  seq_in = words[i:i + seq_length]
  seq_out = words[i + seq_length]
  dataX.append([word for word in seq_in])
  dataY.append(seq_out)
n_patterns = len(dataX)
print("Número de Instâncias: ", n_patterns)

tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(dataX)
X = tokenizer.texts_to_sequences(dataX)
y = tokenizer.texts_to_sequences(dataY)

flat_y = [item for sublist in y for item in sublist]

print('Numero x: ', len(X))
print('Numero y: ', len(flat_y))

X = numpy.reshape(X, (n_patterns, seq_length))
y = np_utils.to_categorical(flat_y)

hidden_size = 128
dropout=0.5
model = Sequential()
model.add(Embedding(n_vocab, 128, input_length=seq_length))     # diminuida a capacidade do modelo reduzindo a camada de embedding de 1024 para 128
model.add(GRU(hidden_size, return_sequences=True, dropout=dropout))
model.add(GRU(hidden_size, dropout=dropout))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=12, batch_size=256, callbacks=callbacks_list)

Número de Instâncias:  99950
Numero x:  99950
Numero y:  99950




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/12





99950/99950 [==============================] - 87s 867us/step - loss: 6.9663

Epoch 00001: loss improved from inf to 6.96627, saving model to weights-improvement-01-6.9663.hdf5
Epoch 2/12
99950/99950 [==============================] - 85s 854us/step - loss: 6.0515

Epoch 00002: loss improved from 6.96627 to 6.05151, saving model to weights-improvement-02-6.0515.hdf5
Epoch 3/12
99950/99950 [==============================] - 86s 858us/step - loss: 5.4679

Epoch 00003: loss improved from 6.05151 to 5.46789, saving model to weights-improvement-03-5.4679.hdf5
Epoch 4/12
99950/99950 [==============================] - 85s 855us/step - loss: 4.9668

Epoch 00004: loss improved from 5.46789 to 4.96684, s

In [6]:
import sys
import numpy

#seleciona um trecho aleatório do livro para servir como entrada
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
X = tokenizer.texts_to_sequences([pattern])[0]
print('Texto fornecido como entrada: ')
print("\"", ' '.join([value for value in pattern]), "\"")
print('Continuação do texto gerado pela rede: ')

probability = 1.0

for i in range(50):
  x_in = numpy.reshape(X, (1, seq_length))
  prediction = model.predict(x_in, verbose=0)
  index = numpy.argmax(prediction)
  probability *= prediction[0][index]

  result = tokenizer.sequences_to_texts([[index]])

  sys.stdout.write(result[0])
  sys.stdout.write(' ')
  X.append(index)
  X = X[1:len(X)]
  
print()
print('Probabilidade do texto gerado: ', probability)

Texto fornecido como entrada: 
" suplemento volume no vii tomo no i v suplemento no 147 publicado no dsf paginas 774 982 pub informacoes complementares suplemento volume no vii tomo no v v suplemento no 147 publicado no dsf paginas 641 771 pub informacoes complementares suplemento volume no vii tomo no iv v suplemento no "
Continuação do texto gerado pela rede: 
197 publicado no dsf paginas 3 280 pub informacoes complementares suplemento volume no iv tomo no iv suplemento no 197 publicado no dsf paginas 3 280 pub informacoes complementares suplemento volume no iv tomo no iv suplemento no 197 publicado no dsf paginas 3 280 pub informacoes complementares suplemento volume 
Probabilidade do texto gerado:  1.1639031646911483e-10


In [7]:
!pip install xlrd

In [8]:
# importando os dois outros conjuntos de dados

import xlrd     # biblioteca para parsing de arquivos .xlsx

class PEC:
  def __init__(self, num="", ass="", just="", res=""):
    self.numero = num
    self.assunto = ass
    self.descricao = just
    self.resultado = res
  def __repr__(self):
    return "Número: %s, Assunto: %s, Resultado: %s, Descricao: %s \n" % (self.numero, self.assunto, self.resultado, self.descricao)

!cp "/content/drive/My Drive/congresso_artificial/PECs.xlsx" "/content/"

xl_workbook = xlrd.open_workbook("/content/PECs.xlsx")
lista_pecs = []

for xl_sheet in xl_workbook.sheets():
  for row_idx in range(1, xl_sheet.nrows):
    temp_pec = []
    for col_idx in range(0, xl_sheet.ncols):
      temp_pec.append(xl_sheet.cell(row_idx, col_idx).value)
    pec = PEC(temp_pec[0], temp_pec[1], temp_pec[2], temp_pec[4])
    lista_pecs.append(pec)

pd.DataFrame(lista_pecs)

,0
0,"Número: Nº: 141/2015, Assunto: Administrativo ..."
1,"Número: Nº: 15/2015, Assunto: Jurídico - Direi..."
2,"Número: Nº 72/2015, Assunto: Econômico - Tribu..."
3,"Número: Nº 61/2015, Assunto: Econômico - Plane..."
4,"Número: Nº 153/2015, Assunto: Social - Meio am..."
...,...
127,"Número: 20/2017, Assunto: Social - Saúde, Resu..."
128,"Número: 29/2017, Assunto: Administrativo - Org..."
129,"Número: 33/2017, Assunto: Jurídico - Direito e..."
130,"Número: 004/2017, Assunto: Administrativo - Se..."


In [9]:
import xml.etree.ElementTree as ET
import unicodedata
import re

class Votacao:
  def __init__(self, data="", cod=0, sigla="", casa="", secreta="", res="", votos={}, total_sim=0, total_nao=0, ano="", desc="", desc_ident=""):
    self.data_sessao = data       # string formato aaaa-mm-dd
    self.codigo_sessao = cod      # inteiro formado por cast de string
    self.sigla_materia = sigla    # string, sigla referente a materia votada (Ex. PEC, MSF)
    self.casa = casa              # string, referente a casa onde foi votada a sessao
    self.secreta = secreta        # string com valores S ou N (secreta ou nao-secreta)
    self.resultado = res          # string com valores A ou R (materia aprovada ou reprovada)
    self.votos = votos            # dicionario que relaciona numero do senador com o valor do voto (sim, nao, ausente, nao registrou, etc)
    self.total_votos_sim = total_sim  # inteiro formado por cast de string
    self.total_votos_nao = total_nao  # inteiro formado por cast de string
    self.ano_materia = ano        # string em formato aaaa
    self.descricao = desc         # string grande com descricao da votacao da materia
    self.descricao_identificacao = desc_ident  # string que identifica as materias pelo indice
  
  def __repr__(self):
    return "Data: %s, Codigo: %s, Resultado: %s \n" % (self.data_sessao, self.codigo_sessao, self.resultado)

lista_votacoes = []               # lista de instancias da classe Votacao, pode ser exportada para outro arquivo

def trata_dado_ausente(elemento):# funcao para tratar objetos do tipo "NoneType" que aparecem quando
  var = elemento                  # ha dados ausentes, quando o elemento e uma string.
  if(elemento is None):           # caso o elemento seja NoneType, atribui-se uma string vazia a ele
    var = ""
  else:                           # caso contrario, ele recebe o conteudo do elemento a que pertence
    var = var.text
  return var

def trata_dado_ausente_int(elemento):  # trata objetos "NoneType" e os substitui por numeros
  var = elemento
  if(elemento is None):
    var = 0
  else:
    var = int(var.text)
  return var

def limpar_texto(texto):          # trata caracteres com acento e caracteres invalidos.
  texto = texto.lower()
  nfkd = unicodedata.normalize('NFKD', texto)
  palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
  texto_limpo = re.sub('[^a-zA-Z0-9 \\\]', ' ', palavraSemAcento)
  texto_limpo = ' '.join(texto_limpo.split())

  return texto_limpo

for i in range(1991, 2018):       # itera sobre os anos dos arquivos XML das votacoes e faz o parsing do xml
  with open("lista_votacoes_xml/lista_votacoes" + str(i) + ".xml", 'r', encoding="utf-8") as f:
    root = ET.fromstring(f.read())

  for votacao_element in root[1].findall("Votacao"):
    data1 = trata_dado_ausente(votacao_element.find("DataSessao"))
    codigo1 = trata_dado_ausente_int(votacao_element.find("CodigoSessao"))
    sigla1 = trata_dado_ausente(votacao_element.find("SiglaMateria"))
    casa1 = trata_dado_ausente(votacao_element.find("SiglaCasa"))
    secreta1 = trata_dado_ausente(votacao_element.find("Secreta"))
    resultado1 = trata_dado_ausente(votacao_element.find("Resultado"))

    votos = {}
    for voto in votacao_element.iter("Votos"):
      codigo_parl = voto.find("CodigoParlamentar")
      voto_parl = voto.find("Voto")
      votos[codigo_parl] = voto_parl

    total_votos_sim1 = trata_dado_ausente_int(votacao_element.find("TotalVotosSim"))
    total_votos_nao1 = trata_dado_ausente_int(votacao_element.find("TotalVotosNao"))
    ano_materia1 = trata_dado_ausente(votacao_element.find("AnoMateria"))
    descricao1 = limpar_texto(trata_dado_ausente(votacao_element.find("DescricaoVotacao")))
    descricao_identificacao1 = trata_dado_ausente(votacao_element.find("DescricaoIdentificacaoMateria"))
    
    instancia_votacao = Votacao(data1, codigo1, sigla1, casa1, secreta1, resultado1, votos, total_votos_sim1, total_votos_sim1, ano_materia1, descricao1)
    lista_votacoes.append(instancia_votacao)    # atribui os valores do arquivo XML as instancias da classe

pd.DataFrame(lista_votacoes)

,0
0,"Data: 1991-03-05, Codigo: 20277, Resultado: A \n"
1,"Data: 1991-03-05, Codigo: 20277, Resultado: A \n"
2,"Data: 1991-03-06, Codigo: 20278, Resultado: A \n"
3,"Data: 1991-03-06, Codigo: 20278, Resultado: A \n"
4,"Data: 1991-03-06, Codigo: 20278, Resultado: A \n"
...,...
3195,"Data: 2017-12-13, Codigo: 57023, Resultado: \n"
3196,"Data: 2017-12-13, Codigo: 57023, Resultado: A \n"
3197,"Data: 2017-12-13, Codigo: 57023, Resultado: A \n"
3198,"Data: 2017-12-13, Codigo: 57023, Resultado: A \n"


In [10]:
# criando dois novos conjuntos de dados. um com votacoes do senado e outras com PECs. cada uma tem target 0 ou 1, significando se foram aprovadas ou nao.
import random

data1 = [] # votacoes
data2 = [] # pecs

for votacao in lista_votacoes:
  if votacao.resultado == "A":
    data1.append([(votacao.descricao), 1])
  elif votacao.resultado == "R":
    data1.append([(votacao.descricao), 0])

random.shuffle(data1)
data1_train = data1[ : int(len(data1)*0.8)]
data1_test = data1[int(len(data1)*0.8) : ]

print(len(data1_train), len(data1_test))

for pec in lista_pecs:
  if "aprovada" in pec.resultado.lower():
    descricao_pec = limpar_texto(pec.descricao)
    data2.append([descricao_pec, 1])
  elif "arquivada" in pec.resultado.lower():
    resultado_pec = limpar_texto(pec.resultado)
    data2.append([pec.resultado, 0])

random.shuffle(data2)
data2_train = data2[ : int(len(data2)*0.8)]
data2_test = data2[int(len(data2)*0.8) : ]

print(len(data2_train), len(data2_test))

2440 610
100 25


In [11]:
# alterando a rede neural ja treinada para treinar no novo conjunto de dados.

new_model = model

for layer in new_model.layers[:3]:    # faz com que as camadas de embedding e de GRUs fiquem congeladas.
    layer.trainable = False

new_model.pop()
new_model.add(Dense(2, activation='softmax'))   # substitui a camada de saida por uma camada binaria, em vez de um vetor de palavras.
new_model.compile(loss='categorical_crossentropy', optimizer='adam')

new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 1024)          10743808  
_________________________________________________________________
gru_1 (GRU)                  (None, 50, 128)           442752    
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 11,318,530
Trainable params: 33,282
Non-trainable params: 11,285,248
_____________________________________

In [0]:
# data2_train_x = [i[0] for i in data1_train]
# for frase in data2_train_x:
#   frase = text_to_word_sequence(frase, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n', split=' ', lower=False)
#   print(frase)
# print(data2_train_x)

In [13]:
data1_train_x = [i[0] for i in data1_train]
data1_train_y = [i[1] for i in data1_train]

data2_train_x = [i[0] for i in data2_train]
data2_train_y = [i[1] for i in data2_train]

# separando as strings inteiras em listas de palavras, e em seguida transformando em tokens
temp = []
for frase in data1_train_x:
  frase_lista = text_to_word_sequence(frase, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n', split=' ', lower=False)
  temp.append(frase_lista)
data1_train_x = temp

temp = []
for frase in data2_train_x:
  frase_lista = text_to_word_sequence(frase, filters='"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n', split=' ', lower=False)
  temp.append(frase_lista)
data2_train_x = temp

tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(data1_train_x)
tokenizer.fit_on_texts(data2_train_x)
data1_train_x = tokenizer.texts_to_sequences(data1_train_x)
data2_train_x = tokenizer.texts_to_sequences(data2_train_x)
data1_train_x = np.array(data1_train_x)
data1_train_x = np.nan_to_num(data1_train_x)
data2_train_x = np.array(data2_train_x)

pd.DataFrame(data1_train_x)

# PROBLEMA: a descricao de PECs, quando vetorizada, fica com dimensoes irregulares pois as descricoes possuem tamanhos diferentes.

# print(data1_train_x.shape)

,0
0,"[10, 2, 12, 158, 1, 149, 1117, 1118, 7, 11, 6,..."
1,"[71, 55, 47, 5, 102, 17, 15, 1, 26, 2, 61, 3, ..."
2,"[8, 57, 4, 36, 2, 12, 591, 2, 503, 149, 28, 1,..."
3,"[8, 2, 101, 350, 31, 42, 42, 13, 54, 2, 31, 42..."
4,"[10, 2, 167, 65, 1, 242, 657, 7, 35, 6, 60, 50..."
...,...
2435,"[8, 4, 37, 3, 608, 1, 85, 10, 1, 72, 1, 75, 78..."
2436,"[8, 4, 175, 288, 1, 85, 10, 1, 72, 1, 75, 78, ..."
2437,"[14, 3, 204, 55, 104, 4, 3802, 14, 17, 3803, 8..."
2438,"[10, 2, 167, 1833, 1834, 1, 242, 13, 197, 174,..."


In [14]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

new_model.fit(data1_train_x, data1_train_y, epochs=12, batch_size=256, callbacks=callbacks_list)

ValueError: ignored